In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
import plotly.express as px
from datetime import datetime
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False    # 负号显示的解决方案


import toml
from pprint import pprint


from dash import Dash, Input, Output
# from grpc import server
import plotly.express as px
import plotly.graph_objs as go
import dash
from dash import dcc                  # 交互式组件
from dash import html                 # 代码转html
from dash.dependencies import Input, Output, State         # 回调
from jupyter_dash import JupyterDash
import json
from dash.exceptions import PreventUpdate
# pip3 install jupyter-server-proxy
# pip3 install jupyter-dash

### __读取数据__

In [118]:
# data = pd.read_excel('./2022年售后维修在线编辑汇总(1).xlsx')
data = pd.read_excel('./2022年售后维修在线编辑汇总(2).xlsx')

### __填补缺失值__

In [119]:
data['规格型号'].fillna('/', inplace=True) #利用'/'填补规格型号的缺失值
print(data['规格型号'].isna().sum())

0


In [120]:
data['序列号ID'].fillna('/', inplace=True) #利用'/'填补序列号的缺失值
print(data['序列号ID'].isna().sum())

0


In [121]:
data['问题现象'].fillna('mask1', inplace=True)
print(data['问题现象'].isna().sum())

0


In [122]:
# data.dropna(axis=0, subset=['问题原因'], inplace=True)
data['问题原因'].fillna('mask2', inplace=True)
print(data['问题原因'].isna().sum())

0


In [123]:
# data.dropna(axis=0, subset=['解决方案'], inplace=True)
data['解决方案'].fillna('mask3', inplace=True)
print(data['解决方案'].isna().sum())

def split_for_first(x):
    y = x.split('/')
    y = y[0]
    return y


data['解决方案'] = data['解决方案'].apply(split_for_first)

0


In [124]:
# 提取产品生产日期
data['序列号ID'] = data['序列号ID'].astype('str')
def split_for_time(x):
    y = x.split('-')
    res = ''
    if len(y)>3:
        # print(y[3])
        if len(y[3])<=8:
            res += y[3]
            # res += datetime.strftime(datetime.strptime(y[3],'%Y%m%d'), '%Y-%m-%d')
        else:
            res += y[3][:8]
            # res += datetime.strftime(datetime.strptime(y[3][:8],'%Y%m%d'), '%Y-%m-%d')
    else:
        # print(y[0])
        res += y[0]
    return res


date = data['序列号ID'].apply(split_for_time)
date.name = '产品日期'

data = pd.concat([data, date], axis=1)

In [125]:
# 用于一些验证
# temp = data[data['规格型号']=='LC-K2050A-IGTFS']
# temp = temp[temp['产品日期']=='20211231']
# temp = temp[temp['项目，客户名称'].str.contains('四川')]
# temp

### __读取配置文件__

In [126]:
# 需要统一的文字描述
Unity_config = toml.load('./UnifiedWriting.toml')
# pprint(Unity_config['unity']['problemappearance'])
# pprint(Unity_config['unity']['problemcause'])
# pprint(Unity_config['unity']['solution'])

# 需要添加 "更新"的配件名
addword_config = toml.load('./AddReplaceWord.toml')
# pprint(addword_config)
needchange_parts = addword_config['needadd_parts']['parts']

### __处理文本信息__

In [127]:
# # 考虑将部分列的英文标点转换成中文标点，并去掉末尾的换行符号

# # 规格型号、问题现象、问题原因、解决方案这几列需要转换
# table = {ord(f):ord(t) for f,t in zip(
#         # u'()!?;[],，',
#         # u'（）！？；【】//'                 #这里顺便把，也换成了/
#         u'(),',
#         u'（），'
#     )
# }
# columnname = ['规格型号', '问题现象', '问题原因', '解决方案']
# cols = []
# for i in columnname:
#     cols.append(list(data.columns).index(i))
# def translate_E2C(cols):
#     for i in range(len(data)):
#         for col in cols:
#             if type(data.iloc[i, col]) == str:
#                 data.iloc[i, col] = data.iloc[i, col].strip('\n') #去掉换行
#                 data.iloc[i, col] = data.iloc[i, col].translate(table)
# translate_E2C(cols)

In [128]:
# 考虑将部分列的英文标点转换成中文标点，并去掉末尾的换行符号
# 利用 DataFrame的 apply方法而不是for循环，速度很快

data[['规格型号', '问题现象', '问题原因', '解决方案']].astype('str')
table = {ord(f):ord(t) for f,t in zip(
        u'()!?;[],',
        u'（）！？；【】，'
    )
}
def translate_E2C(x):
    x = x.strip('\n')
    x = x.translate(table)
    # print(x)
    return x


columnname = ['规格型号', '问题现象', '问题原因', '解决方案']
for i in columnname:
    data[i] = data[i].apply(translate_E2C)
# data[data['解决方案'].str.contains('，')]['解决方案'].value_counts()

In [129]:
# 考虑将解决方案列中 UF/V 和 A/V 中的/删去

def convert_UFV():
    temp = data[data['解决方案'].str.contains('UF/') & 
                    data['解决方案'].str.contains('V')]['解决方案']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], list(data.columns).index('解决方案')]
        # print(list(a)[0])
        ind = list(a)[0].index('UF/')
        # print(ind)
        data.iloc[data.index==temp.index[i], list(data.columns).index('解决方案')] = list(a)[0][:ind+2] + list(a)[0][ind+3:] #用字符串切片的方式进行删除
def convert_AV():
    temp = data[data['解决方案'].str.contains('A/') & 
                    data['解决方案'].str.contains('V')]['解决方案']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], list(data.columns).index('解决方案')]
        # print(list(a)[0])
        ind = list(a)[0].index('A/')
        # print(ind)
        data.iloc[data.index==temp.index[i], list(data.columns).index('解决方案')] = list(a)[0][:ind+1] + list(a)[0][ind+2:] #用字符串切片的方式进行删除


convert_UFV()
convert_AV()
data[data['解决方案'].str.contains('UF/') & 
        data['解决方案'].str.contains('V')]['解决方案'].value_counts() #验证转换完毕
data[data['解决方案'].str.contains('A/') & 
        data['解决方案'].str.contains('V')]['解决方案'].value_counts() #验证转换完毕

Series([], Name: 解决方案, dtype: int64)

In [144]:
data[data['解决方案'].str.contains('A') & 
        data['解决方案'].str.contains('V')]['解决方案'].value_counts() #验证转换完毕

更换2A250V保险管               233
更换1.5A250V保险管              74
补2A250V保险管                 35
补1.5A250V保险管               11
更换保险管1.5A250V               9
更换12V开关电源JWAK-18S12WKT      8
更换3.3V转换芯片AMS1117-3.3       4
补保险管2A250V                  4
更换5A250V保险管                 3
更换5V开关电源JWAK-15S5KT         3
更换10A250V保险管*2个             2
更换保险管套与2A250V保险管            2
补2A250V保险管和保险座子             2
更换LVC245A                   2
更换92TT1AIVPQ4               2
更换EE13-12V1A—200uH          1
更换TPA3255DDVR               1
更换24V2A电源适配器                1
更换保险管2A250V                 1
更换2A205V保险管                 1
更换10A250V保险管                1
安装2A250V保险管                 1
Name: 解决方案, dtype: int64

In [130]:
# 考虑将问题现象列的 1.2.转换为/形式，统一一下

def convert_123():
    temp = data[data['问题现象'].str.startswith('1.')]['问题现象']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], list(data.columns).index('问题现象')]
        # print(list(a)[0])
        if list(a)[0][:2]=='1.': #这句 if判断没有也可以
            b = list(a)[0][2:].split('.') #利用 '.'对字符串进行分割
            for j in range(len(b)):
                if j < len(b)-1: #这一步是删除经过 split分割后的多余数字
                    b[j] = b[j][:-1]
            data.iloc[data.index==temp.index[i], list(data.columns).index('问题现象')] = '/'.join(b)
            # print(data.iloc[data.index==temp.index[i], 8].tolist()[0])
convert_123()


data[data['问题现象'].str.startswith('1.')]['问题现象'].value_counts() #验证转换完毕

Series([], Name: 问题现象, dtype: int64)

In [131]:
# 考虑部分意义相同的情况，需要进行修改，统一改成相同的表述
# 因为有些数据是形如：不上电/网络连接不上
#                   不上电-网络连接不上

def handle_backslash_123(data, temp, changedstr, judgestr='', col='问题现象'):
    '''
        data: 是一个 pandas的 DataFrame
        temp: data经过 .str.contains()筛选后的结果
        judgestr: 判断字符, 例如网络连接不上的判断字符就是 '网络', 有时候不存在 '/'以及 '1.2.3.'的情况也可以选择不传入
        changedstr: 修改后的字符, 例如网络连接不上的修改后的字符就是 '网络不通'
        i: temp的下标, 因为需要遍历来按照索引进行修改
    '''
    if col != '问题现象':
        loc = data.columns.tolist().index(col)
    else:
        loc = data.columns.tolist().index('问题现象') #找到问题现象出现的列索引
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], loc]
        if '/' in list(a)[0] or '，' in list(a)[0]: #多问题情况下的统一表述
            # print(list(a)[0])
            if '/' in list(a)[0]:
                split_str = '/'
            elif '，' in list(a)[0]:
                split_str = '，'
            b = list(a)[0].split(split_str)
            for j in range(len(b)):
                if judgestr in b[j]:
                    b[j] = changedstr
                    break
            data.iloc[data.index==temp.index[i], loc] = split_str.join(b)
        else:
            data.iloc[data.index==temp.index[i], loc] = changedstr

In [132]:
cols = ['问题现象', '问题原因', '解决方案']
Englishcol = ['problemappearance', 'problemcause', 'solution']
cols_dict = dict(zip(cols, Englishcol))

def Unity(col, temp=[]):
    keys = list(Unity_config['unity'][cols_dict[col]]['or'].keys())
    for i in keys:
        print("-----", i, ": ")
        if Unity_config['unity'][cols_dict[col]]['or'][i]:
            screen = '|'.join(Unity_config['unity'][cols_dict[col]]['or'][i])
            # print(screen)
            temp = data[col].str.contains(screen)
        if Unity_config['unity'][cols_dict[col]]['and'][i]:
            for j in Unity_config['unity'][cols_dict[col]]['and'][i]:
                # print(j)
                # print(len(temp))
                if len(temp)!=0:
                    if j[0] == '~':
                        temp &= ~data[col].str.contains(j[1:])
                    else:
                        temp &= data[col].str.contains(j)
                else:
                    if j[0] == '~':
                        temp = ~data[col].str.contains(j[1:])
                    else:
                        temp = data[col].str.contains(j)
        temp = data[temp][col]
        # print(temp)
        handle_backslash_123(data, temp, changedstr=i,
                            judgestr=Unity_config['unity'][cols_dict[col]]['str'][i], col=col)
        temp = []

Unity(col='问题现象')
Unity(col='问题原因')
Unity(col='解决方案')

----- 不上电 : 
----- 4G不在线 : 
----- 无声音输出 : 
----- 网络不通 : 
----- 声音异常 : 
----- 搜索不到终端 : 
----- 不出串口 : 
----- 电源指示灯不亮 : 
----- 网络指示灯不亮 : 
----- 不断重启 : 
----- 检测无问题 : 
----- 开关电源坏 : 
----- 通讯模块坏 : 
----- 功放芯片坏 : 
----- 网络芯片坏 : 
----- SIM卡槽坏 : 
----- FB2电感坏 : 
----- 升级程序 : 
----- 更换网口座子RJ45 : 
----- 更换功放芯片MP7770 : 


In [133]:
cols = ['问题现象', '问题原因', '解决方案']
Englishcol = ['problemappearance', 'problemcause', 'solution']
cols_dict = dict(zip(cols, Englishcol))

def Unity(col, temp=[]):
    keys = list(Unity_config['unity'][cols_dict[col]]['or'].keys())
    for i in keys:
        print("-----", i, ": ")
        if Unity_config['unity'][cols_dict[col]]['or'][i]:
            screen = '|'.join(Unity_config['unity'][cols_dict[col]]['or'][i])
            # print(screen)
            temp = data[col].str.contains(screen)
        if Unity_config['unity'][cols_dict[col]]['and'][i]:
            for j in Unity_config['unity'][cols_dict[col]]['and'][i]:
                # print(j)
                # print(len(temp))
                if len(temp)!=0:
                    if j[0] == '~':
                        temp &= ~data[col].str.contains(j[1:])
                    else:
                        temp &= data[col].str.contains(j)
                else:
                    if j[0] == '~':
                        temp = ~data[col].str.contains(j[1:])
                    else:
                        temp = data[col].str.contains(j)
        temp = data[temp][col]
        # print(temp)
        handle_backslash_123(data, temp, changedstr=i,
                            judgestr=Unity_config['unity'][cols_dict[col]]['str'][i], col=col)
        temp = []

Unity(col='问题现象')
Unity(col='问题原因')
Unity(col='解决方案')

----- 不上电 : 
----- 4G不在线 : 
----- 无声音输出 : 
----- 网络不通 : 
----- 声音异常 : 
----- 搜索不到终端 : 
----- 不出串口 : 
----- 电源指示灯不亮 : 
----- 网络指示灯不亮 : 
----- 不断重启 : 
----- 检测无问题 : 
----- 开关电源坏 : 
----- 通讯模块坏 : 
----- 功放芯片坏 : 
----- 网络芯片坏 : 
----- SIM卡槽坏 : 
----- FB2电感坏 : 
----- 升级程序 : 
----- 更换网口座子RJ45 : 
----- 更换功放芯片MP7770 : 


In [134]:
# 处理多问题现象的情况

def split_multiple(problem, cnt):
    '''
        主要用于分离含有 '/'或者 '，'的问题现象
        注意要先忽略掉 '/升级程序', 再分离
    '''
    i = 0
    while i < len(problem):
        # 先处理含有多情况下的升级程序情况
        if '/升级程序' in problem[i] or "升级程序/" in problem[i]:
            if '/升级程序' in problem[i]:
                problem[i] = ''.join(problem[i].split('/升级程序'))
            if '升级程序/' in problem[i]:
                problem[i] = ''.join(problem[i].split('升级程序/'))
        if '，升级程序' in problem[i] or "升级程序，" in problem[i]:
            if '，升级程序' in problem[i]:
                problem[i] = ''.join(problem[i].split('，升级程序'))
            if '升级程序，' in problem[i]:
                problem[i] = ''.join(problem[i].split('升级程序，'))
        # 再处理正常情况
        if '/' in problem[i] or '，' in problem[i]:
            if '/' in problem[i]:
                temp = problem[i].split('/')
            elif '，' in problem[i]:
                temp = problem[i].split('，')
            problem += temp
            cnt += [cnt[i]]*len(temp)
            problem.pop(i)
            cnt.pop(i)
        else:
            i += 1

def sum_multiple(problem, cnt):
    '''
        有时候存在一个规格型号有多个问题现象, 那可能需要和之前单一的情况合并起来考虑
        problem: 是一个列表, 存放着需要绘制图像的文本数据
        cnt: 是一个列表, 存放着需要绘制图像的数值数据
        先根据文本长度排序, 短的文本在前面, 防止丢失不一样的解决方案
        有时候存在空字符串 '', 需要删掉
        之后去重
        注意在 needchange_parts中的配件需要添加更换二字
    '''
    temp = sorted(enumerate(problem), key=lambda x:len(x[1]), reverse=False)
    idx = [i[0] for i in temp]
    problem = [i[1] for i in temp]
    cnt = [cnt[i] for i in idx]

    # 去重
    pro_dict = {}
    for i in range(len(problem)):
        if problem[i] != '': #忽略 ''的情况
            if problem[i] in needchange_parts:
                problem[i] = '更换'+problem[i]
            flag = 0
            # # 对于含有相同文本的情况，只取最长的文本
            # for key in pro_dict.keys(): #这一步是在去重
            #     if problem[i] in key: #短的文本在长的里面
            #         pro_dict[key] += cnt[i] #长的值增加
            #         flag = 1
            #         break
            if flag == 0:
                if problem[i] not in pro_dict:
                    pro_dict[problem[i]] = cnt[i]
                else:
                    pro_dict[problem[i]] += cnt[i]
    
    # print(list(pro_dict.keys()), '\n', list(pro_dict.values()))
    # problem, cnt = list(pro_dict.keys()), list(pro_dict.values())
    return list(pro_dict.keys()), list(pro_dict.values()) #字典解包

In [135]:
# 记录 '成品' 与 '板件' 的型号
product_type = list(data['产品类型'].value_counts().index)[:-1]

model = {} #字典用于存放成品或者板件的型号
for i in product_type:
    model[i] = list(data[data['产品类型']==i]['规格型号'].value_counts().index)

In [136]:
# # options、value、children？？？
# app = JupyterDash('Chained Callbacks')
# all_options = model

# app.layout = html.Div([
#     html.Div(
#         [html.H4('售后服务数据')]
#     ),
#     html.Div(
#         [dcc.Graph(id = "graph")],
#         # style = dict(width = '64%', display = 'inline-block')
#     ),
#     html.Div(
#         [html.P("产品类型:"),
#         dcc.Dropdown(id = 'types',
#                     options = [{'label': k, 'value': k} for k in all_options.keys()],
#                     value = '成品', #初始值为成品
#                     clearable = False
#         ),
#         html.P("规格型号:"),
#         dcc.Dropdown(id = 'specifications',
#                     clearable = False
#         ),
#         html.P("列名:"),
#         dcc.Dropdown(id = 'columns',
#                     options = ['问题现象', '问题原因', '解决方案'],
#                     value = '问题现象', #初始值为问题现象
#                     clearable = False
#         )],
#         style = dict(width = '40%', display = 'inline-block') #设置下拉列表的大小
#     )
# ])

# @app.callback(
#     Output('specifications', 'options'), #options即下面的return是一个列表嵌套一个字典
#     [Input('types', 'value')])
# def set_type_options(select_type): #select_type是传入的参数，表示选择成品还是组件
#     return [{'label': i, 'value': i} for i in all_options[select_type]]

# @app.callback(
#     Output('specifications', 'value'),
#     [Input('specifications', 'options')])
# def set_specification_value(available_options): #available_options表示上面返回的列表嵌套字典
#     return available_options[0]['value'] #取列表的第一项，找到字典键为value对应的值，这里选择键label也一样的


# @app.callback(
#     Output("graph", "figure"), #需要更新的输出，即图像
#     [Input("types", "value"), #需要更新的输入，有三个：产品类型、规格型号、列名
#     Input("specifications", "value"),
#     Input("columns", "value"),])
# def generate_chart(types, specifications, columns):
#     texts = []
#     values = []
#     i = len(texts)
#     temp = data[data['规格型号']==specifications][columns].value_counts()

#     texts.append(list(temp.index))
#     values.append(list(temp))

#     # 是否忽略缺失值
#     def check_exist(textdata):
#         ind = []
#         check_str = ['mask1', 'mask2', 'mask3', '检测无问题']
#         for str in check_str:
#             if str in textdata:
#                 ind.append(textdata.index(str))
#         return ind
#     ind = sorted(check_exist(texts[i]), reverse=True)
#     for j in ind:
#         texts[i].pop(j)
#         values[i].pop(j)

#     split_multiple(texts[i], values[i])
#     texts[i], values[i] = sum_multiple(texts[i], values[i])

#     labels, numsdata = texts[i], values[i]
#     df = pd.DataFrame(zip(labels, numsdata), columns=[columns, '次数'])
#     fig = px.pie(
#                 df,
#                 values = '次数',
#                 names = columns,
#                 hole = 0.3,
#                 title = '型号 '+ specifications +' 的主要'+ columns,
#     )
#     fig.update_traces(textposition='inside')
#     fig.update_layout(uniformtext_minsize=10, 
#                         uniformtext_mode='hide',)
#     return fig


# if __name__ == '__main__':
#     # app.run_server(mode='inline', width='80%', height=800)
#     app.run_server(mode='external', host='127.0.0.1', port=8050, debug=True)

[http://127.0.0.1:8050/](http://127.0.0.1:8050/)

In [137]:
# .ipynb文件转.py文件：
# !jupyter nbconvert --to python 。。。.ipynb

In [138]:
# 端口被占用时：
# !netstat -ano|findstr 8050
# !taskkill/F /pid ...

### 尝试划分子图

In [139]:
# # options、value、children？？？
# app = JupyterDash('Chained Callbacks')
# all_options = model

# app.layout = html.Div([
#     html.Div(
#         [html.H4('售后服务数据')]
#     ),
#     html.Div(
#         [html.P("产品类型:"),
#         dcc.Dropdown(id = 'types',
#                     options = [{'label': k, 'value': k} for k in all_options.keys()],
#                     value = '成品', #初始值为成品
#                     clearable = False
#         ),
#         html.P("规格型号:"),
#         dcc.Dropdown(id = 'specifications',
#                     clearable = False
#         ),
#         html.P("列名:"),
#         dcc.Dropdown(id = 'columns',
#                     options = ['问题现象'],
#                     value = '问题现象', #初始值为问题现象
#                     clearable = False
#         )],
#         style = dict(width = '30%', display = 'inline-block', float = 'right', padding = '80px 0px 0px 0px') #设置下拉列表的大小
#                                                                                                                 # padding是上左下右
#     ),
#     html.Div([
#             dcc.Graph(id = "graph"),
#         ],
#         style = dict(width = '70%', display = 'inline-block',)
#     ),
#     html.Div([
#             dcc.Graph(id = "该现象的问题原因"), 
#             html.Div(id = "output1"),
#             html.Div(id = "a")
#         ],
#         style = dict(width = '50%', display = 'inline-block',)
#     ), 
#     html.Div([
#             dcc.Graph(id = "该现象的解决方案"),
#             html.Div(id = "output2")
#         ],
#         style = dict(width = '50%', display = 'inline-block', float='right')
#     ),    
# ])

# # 选择产品类型
# @app.callback(
#     Output('specifications', 'options'), #options即下面的return是一个列表嵌套一个字典
#     [Input('types', 'value')])
# def set_type_options(select_type): #select_type是传入的参数，表示选择成品还是组件
#     return [{'label': i, 'value': i} for i in all_options[select_type]]

# # 选择规格型号
# @app.callback(
#     Output('specifications', 'value'),
#     [Input('specifications', 'options')])
# def set_specification_value(available_options): #available_options表示上面返回的列表嵌套字典
#     return available_options[0]['value'] #取列表的第一项，找到字典键为value对应的值，这里选择键label也一样的

# # 根据问题现象来画图
# @app.callback(
#     Output("graph", "figure"), #需要更新的输出，即图像
#     [Input("types", "value"), #需要更新的输入，有三个：产品类型、规格型号、列名
#     Input("specifications", "value"),
#     Input("columns", "value"),])
# def generate_chart(types, specifications, columns):
#     texts = []
#     values = []
#     i = len(texts)
#     temp = data[data['规格型号']==specifications][columns].value_counts()

#     texts.append(list(temp.index))
#     values.append(list(temp))

#     # 是否忽略缺失值
#     def check_exist(textdata):
#         ind = []
#         check_str = ['mask1', 'mask2', 'mask3', '检测无问题']
#         for str in check_str:
#             if str in textdata:
#                 ind.append(textdata.index(str))
#         return ind
#     ind = sorted(check_exist(texts[i]), reverse=True)
#     for j in ind:
#         texts[i].pop(j)
#         values[i].pop(j)

#     # split_multiple(texts[i], values[i])
#     # texts[i], values[i] = sum_multiple(texts[i], values[i])

#     labels, numsdata = texts[i], values[i]
#     df = pd.DataFrame(zip(labels, numsdata), columns=[columns, '次数'])
    

#     ind = df.sort_values(by='次数', inplace=False, ascending=False)['次数'].index.tolist()
#     # print(ind)
#     colors = ['lightslategray',] * len(df)
#     if len(ind) >= 10:
#         length = 3
#     else:
#         length = 1
#     for i in range(length):
#         colors[ind[i]] = 'crimson'
#     fig = go.Figure(
#         data=[
#             go.Bar(
#                 x = df['次数'],
#                 y = df[columns],
#                 text = df['次数'],
#                 marker_color = colors,
#                 orientation = 'h', #横向的条形图
#             )
#         ]
#     )
#     fig.update_traces(texttemplate='%{text:.s}', textposition='outside')
#     fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
#                         title_text='型号 '+ specifications +' 的主要'+ columns,)
#     return fig

# # 可视化某一问题现象对应的问题原因
# @app.callback(
#     [Output("该现象的问题原因", "figure"), #需要更新的输出，即图像
#     Output("a", "children")], #输出当前的问题原因
#     # Output("output1", "children"), #可以查看输出结果
#     [Input("graph", "hoverData"),
#     State("specifications", "value"),
#     State("columns", "value")])
# def synchronization_1(hoverdata, specifications, columns):
#     a = str(hoverdata)
#     if columns == '问题现象' and a:
#         a = a[60:].split(",")[2].split(":")[1].split("'")[1]
#         # return a[60:].split(",")[2].split(":")[1].split("'")[1]
#         temp = data[data['规格型号']==specifications]
#         temp = temp[temp['问题现象']==a]['问题原因'].value_counts()
#         texts = list(temp.index)
#         values = list(temp)
#         dff = pd.DataFrame(zip(texts, values), columns=['问题原因', '次数'])
#         fig = px.pie(
#                 dff,
#                 values = '次数',
#                 names = '问题原因',
#                 hole = 0.3,
#         )
#         fig.update_traces(textposition='inside')
#         fig.update_layout(uniformtext_minsize=10, 
#                             uniformtext_mode='hide',)
#         #                     title_text='问题现象 '+ a +' 的主要问题原因')
#         return fig, '问题现象 '+ a +' 的主要问题原因'
#     else:
#         raise PreventUpdate

# # 可视化某一问题原因对应的解决方案
# @app.callback(
#     Output("该现象的解决方案", "figure"), #需要更新的输出，即图像
#     # Output("output2", "children"), #可以查看输出结果
#     [Input("该现象的问题原因", "hoverData"),
#     State("specifications", "value"),
#     State("columns", "value"),
#     State("a", "children")])
# def synchronization_2(hoverdata, specifications, columns, a):
#     b = str(hoverdata)
#     if columns == '问题现象' and b:
#         # a = a[60:].split(",")[2].split(":")[1].split("'")[1]
#         b = b[10:].split(",")[1].split(":")[1].split("'")[1]
#         # return b
#         temp = data[data['规格型号']==specifications]
#         temp = temp[temp['问题现象']==a[5:-8]]
#         temp = temp[temp['问题原因']==b]['解决方案'].value_counts()
#         texts = list(temp.index)
#         values = list(temp)
#         dff = pd.DataFrame(zip(texts, values), columns=['解决方案', '次数'])
#         fig = go.Figure(
#             data=[
#                 go.Scatter(
#                     x = dff['解决方案'],
#                     y = dff['次数'],
#                     text = dff['次数'],
#                     mode='lines+markers',
#                     name='lines+markers',
#                 )
#             ]
#         )
#         fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
#                             title_text='问题原因 '+ b +' 的主要解决方案')
#         return fig
#     else:
#         raise PreventUpdate


# if __name__ == '__main__':
#     # app.run_server(mode='inline', width='80%', height=800)
#     app.run_server(mode='external', host='127.0.0.1', port=8060, debug=True)

[http://127.0.0.1:8060/](http://127.0.0.1:8060/)

In [140]:
# 所有型号
specifications = data['规格型号'].value_counts().index
# specifications

# 所有型号对应的生产日期
dates_dict = {}
for i in specifications:
    available_dates = sorted(list(set(data[data['规格型号']==i]['产品日期'])), key=lambda x:x)
    dates_dict[i] = available_dates
# dates_dict

### 添加日期、项目

In [141]:
app = JupyterDash('Chained Callbacks')
all_options = model


app.layout = html.Div([
    html.Div(
        [html.H1('售后服务数据')]
    ),
    html.Div(
        [html.P("产品类型:"),
        dcc.Dropdown(id = 'types',
                    options = [{'label': k, 'value': k} for k in all_options.keys()],
                    value = '成品', #初始值为成品
                    clearable = False
        ),
        html.P("规格型号:"),
        dcc.Dropdown(id = 'specifications',
                    clearable = False
        ),
        html.P("日期:"),
        dcc.Dropdown(id = 'dates',
                    clearable = True
        ),
        html.P("项目:"),
        dcc.Dropdown(id = 'projects',
                    clearable = True
        ),
        html.P("列名:"),
        dcc.Dropdown(id = 'columns',
                    options = ['问题现象'],
                    value = '问题现象', #初始值为问题现象
                    clearable = False
        )],
        style = dict(width = '30%', display = 'inline-block', float = 'right', padding = '0px 0px 0px 0px') #设置下拉列表的大小
                                                                                                                # padding是上左下右
    ),
    html.Div([
            dcc.Graph(id = "graph"),
        ],
        style = dict(width = '70%', display = 'inline-block',)
    ),
    html.Div([
            dcc.Graph(id = "该现象的问题原因"), 
            html.Div(id = "output1"),
            html.Div(id = "a")
        ],
        style = dict(width = '50%', display = 'inline-block',)
    ), 
    html.Div([
            dcc.Graph(id = "该现象的解决方案"),
            html.Div(id = "output2")
        ],
        style = dict(width = '50%', display = 'inline-block', float='right')
    ),
])

# 选择产品类型，返回规格型号的可选项
@app.callback(
    Output('specifications', 'options'), #options即下面的return是一个列表嵌套一个字典
    [Input('types', 'value')])
def set_specification_options(select_type): #select_type是传入的参数，表示选择成品还是组件
    return [{'label': i, 'value': i} for i in all_options[select_type]]
# 选择规格型号
@app.callback(
    Output('specifications', 'value'),
    [Input('specifications', 'options')])
def set_specification_value(available_options): #available_options表示上面返回的列表嵌套字典
    return available_options[0]['value'] #取列表的第一项，找到字典键为value对应的值，这里选择键label也一样的

# 选择生产日期
@app.callback(
    Output('dates', 'options'),
    [Input('specifications', 'value')])
def set_date_options(selected_spe):
    return [{'label': i, 'value': i} for i in dates_dict[selected_spe]]
@app.callback(
    Output('dates', 'value'),
    [Input('specifications', 'value'),
    Input('dates', 'options')])
def set_date_value(selected_spe, date_options):
    return date_options[0]['value']

# 选择项目
@app.callback(
    Output('projects', 'options'),
    [Input('specifications', 'value'),
    Input('dates', 'value')])
def set_project_value(selected_spe, selected_date):
    if selected_spe is None:
        raise PreventUpdate
    temp = data[data['规格型号']==selected_spe]
    temp = temp[temp['产品日期']==selected_date]
    pro_options = list(set(temp['项目，客户名称']))
    return [{'label': i, 'value': i} for i in pro_options]
@app.callback(
    Output('projects', 'value'),
    [Input('specifications', 'value'),
    Input('dates', 'value'),
    Input('projects', 'options')])
def set_project_value(selected_spe, selected_date, pro_options):
    if selected_spe is None:
        raise PreventUpdate
    return pro_options[0]['value']

# 根据问题现象来画图
@app.callback(
    Output("graph", "figure"), #需要更新的输出，即图像
    [Input("types", "value"), #需要更新的输入，有三个：产品类型、规格型号、列名
    Input("specifications", "value"),
    Input("dates", "value"),
    Input("projects", "value"),
    Input("columns", "value"),],
    prevent_initial_call = True)
def generate_chart(types, specifications, dates, projects, columns):
    texts = []
    values = []
    i = len(texts)
    temp = data[data['规格型号']==specifications]
    if temp is None:
        raise PreventUpdate
    else:
        if dates and projects:
            temp = temp[temp['产品日期']==dates]
            temp = temp[temp['项目，客户名称']==projects]
        elif dates:
            temp = temp[temp['产品日期']==dates]

        temp = temp[columns].value_counts()
        texts.append(list(temp.index))
        values.append(list(temp))

        # # 是否忽略缺失值
        # def check_exist(textdata):
        #     ind = []
        #     check_str = ['mask1', 'mask2', 'mask3', '检测无问题']
        #     for str in check_str:
        #         if str in textdata:
        #             ind.append(textdata.index(str))
        #     return ind
        # ind = sorted(check_exist(texts[i]), reverse=True)
        # for j in ind:
        #     texts[i].pop(j)
        #     values[i].pop(j)

        # split_multiple(texts[i], values[i])
        # texts[i], values[i] = sum_multiple(texts[i], values[i])

        labels, numsdata = texts[i], values[i]
        df = pd.DataFrame(zip(labels, numsdata), columns=[columns, '次数'])
        

        ind = df.sort_values(by='次数', inplace=False, ascending=False)['次数'].index.tolist()
        # print(ind)
        colors = ['lightslategray',] * len(df)
        if len(ind) >= 10:
            length = 3
        else:
            length = 1
        for i in range(length):
            colors[ind[i]] = 'crimson'
        fig = go.Figure(
            data=[
                go.Bar(
                    x = df['次数'],
                    y = df[columns],
                    text = df['次数'],
                    marker_color = colors,
                    orientation = 'h', #横向的条形图
                )
            ]
        )
    fig.update_traces(texttemplate='%{text:.s}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                        title_text='型号 '+ specifications +' 的主要'+ columns,)
    return fig

# 可视化某一问题现象对应的问题原因
@app.callback(
    [Output("该现象的问题原因", "figure"), #需要更新的输出，即图像
    Output("a", "children")], #输出当前的问题原因
    # Output("output1", "children"), #可以查看输出结果
    [Input("graph", "hoverData"),
    State("specifications", "value"),
    State("dates", "value"),
    State("projects", "value"),
    State("columns", "value")],
    prevent_initial_call = True)
def synchronization_1(hoverdata, specifications, dates, projects, columns):
    a = str(hoverdata)
    if a is None:
        raise PreventUpdate
    if columns == '问题现象' and a:
        a = a[60:].split(",")[2].split(":")[1].split("'")[1]
        # return a[60:].split(",")[2].split(":")[1].split("'")[1]
        temp = data[data['规格型号']==specifications]
        if dates and projects:
            temp = temp[temp['产品日期']==dates]
            temp = temp[temp['项目，客户名称']==projects]
        elif dates:
            temp = temp[temp['产品日期']==dates]
        temp = temp[temp['问题现象']==a]['问题原因'].value_counts()

        texts = list(temp.index)
        values = list(temp)
        dff = pd.DataFrame(zip(texts, values), columns=['问题原因', '次数'])
        fig = px.pie(
                dff,
                values = '次数',
                names = '问题原因',
                hole = 0.3,
        )
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',)
    #                     title_text='问题现象 '+ a +' 的主要问题原因')
    return fig, '问题现象 '+ a +' 的主要问题原因'

# 可视化某一问题原因对应的解决方案
@app.callback(
    Output("该现象的解决方案", "figure"), #需要更新的输出，即图像
    # Output("output2", "children"), #可以查看输出结果
    [Input("该现象的问题原因", "hoverData"),
    State("specifications", "value"),
    State("dates", "value"),
    State("projects", "value"),
    State("columns", "value"),
    State("a", "children")],
    prevent_initial_call = True)
def synchronization_2(hoverdata, specifications, dates, projects, columns, a):
    b = str(hoverdata)
    if b is None:
        raise PreventUpdate
    if columns == '问题现象' and b:
        # a = a[60:].split(",")[2].split(":")[1].split("'")[1]
        b = b[10:].split(",")[1].split(":")[1].split("'")[1]
        # return b
        temp = data[data['规格型号']==specifications]
        if dates and projects:
            temp = temp[temp['产品日期']==dates]
            temp = temp[temp['项目，客户名称']==projects]
        elif dates:
            temp = temp[temp['产品日期']==dates]

        temp = temp[temp['问题现象']==a[5:-8]]
        temp = temp[temp['问题原因']==b]['解决方案'].value_counts()

        texts = list(temp.index)
        values = list(temp)
        if texts is None:
            return None
        dff = pd.DataFrame(zip(texts, values), columns=['解决方案', '次数'])
        fig = go.Figure(
            data=[
                go.Scatter(
                    x = dff['解决方案'],
                    y = dff['次数'],
                    text = dff['次数'],
                    mode='lines+markers',
                    name='lines+markers',
                )
            ]
        )
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                        title_text='问题原因 '+ b +' 的主要解决方案')
    return fig


if __name__ == '__main__':
    # app.run_server(mode='inline', width='80%', height=800)
    app.run_server(mode='external', host='127.0.0.1', port=8060, debug=True)

Dash app running on http://127.0.0.1:8060/


[http://127.0.0.1:8060/](http://127.0.0.1:8060/)